# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235641395600                   -0.50    7.0         
  2   -7.250358531870       -1.83       -1.40    1.0   9.44ms
  3   -7.251153973371       -3.10       -2.12    1.0   9.49ms
  4   -7.251179743653       -4.59       -2.03    2.0   11.9ms
  5   -7.251332743012       -3.82       -2.67    1.0   9.48ms
  6   -7.251337901105       -5.29       -3.11    1.0   9.60ms
  7   -7.251338640853       -6.13       -3.42    3.0   12.8ms
  8   -7.251338775061       -6.87       -3.94    1.0   10.2ms
  9   -7.251338796734       -7.66       -4.24    3.0   13.7ms
 10   -7.251338798103       -8.86       -4.59    1.0   10.4ms
 11   -7.251338798606       -9.30       -5.19    1.0   10.4ms
 12   -7.251338798691      -10.07       -5.42    3.0   13.5ms
 13   -7.251338798702      -10.94       -5.84    1.0   10.1ms
 14   -7.251338798704      -11.89       -5.98    3.0   13.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06141638429272897
[ Info: Arnoldi iteration step 2: normres = 0.41364139983249515
[ Info: Arnoldi iteration step 3: normres = 0.661594467336248
[ Info: Arnoldi iteration step 4: normres = 0.5250827351373899
[ Info: Arnoldi iteration step 5: normres = 0.2745337464300441
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.08e-02, 3.29e-02, 1.56e-01, 2.13e-01, 2.99e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5529703544248942
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.70e-02, 9.35e-02, 5.28e-01, 9.19e-02, 8.63e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09469380005511173
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.53e-03, 4.32e-02, 1.06e-02, 2.90e-02, 6.84e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08022974824771037
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.28e-0